<a href="https://colab.research.google.com/github/10udCryp7/TV-command-synthesis/blob/main/src_prototype/Phase1_TextGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from openai import OpenAI
import os
from google.colab import userdata


class TextGenerator:
  def __init__(self, model, temperature, export_dir):
    self.model = model
    self.temperature = temperature # sau có thể sửa thành 1 file config

    try:
      api_key = get_api_key("colab")
      self.client = OpenAI(api_key=api_key)

    except Exception as e:
      print(f'fail at init model!')
      raise e

  def get_api_key(env_type: str):
    match env_type:
      case "colab":
        return userdata.get("OPENAI_API_KEY")
      case "local":
        return os.getenv("OPENAI_API_KEY")
      # add more case there

  def generate(self, prompt):
    # just for openai
    response = self.client.response.create(
        model=self.model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=self.temperature,
    )
    return response.output[0].content[0].text

  def create_df(parsed_data):
    rows = []

    for main_type, items in parsed_data.items():
        for idx, item in enumerate(items):
            if isinstance(item, dict):
                # Structured format with segments
                text = item.get('text', '')
                segments = item.get('segments', [])

                rows.append({
                    'text': text,
                    'type': main_type,
                    'segments': segments if segments else None,
                    'has_segments': len(segments) > 0
                })
            else:
                # Simple string format (fallback)
                rows.append({
                    'text': item,
                    'type': main_type,
                    'segments': None,
                    'has_segments': False
                })

    df = pd.DataFrame(rows)

    return df



In [ ]:
# utils.py

import json
import re

def clean_json_string(raw_output):
    # Bỏ phần ```json và ```
    cleaned = re.sub(r"^```json\s*|\s*```$", "", raw_output.strip())
    return cleaned

def parse_generated_json(raw_output):
    cleaned = clean_json_string(raw_output)
    return json.loads(cleaned)
